# 쇼핑몰 리뷰 평점 분류 경진대회

## 1. 명령 인자 입력
#### 필자는 .py파일로 터미널에서 학습을 하였기 때문에 .ipynb형태로 변형

In [34]:
import argparse
p = argparse.ArgumentParser()

p.add_argument('--model_fn', required=False)
p.add_argument('--train_fn', required=False)
p.add_argument('--pretrained_model_name', type=str, default="beomi/KcELECTRA-base")  
p.add_argument('--gpu_id', type=int, default=-1)
p.add_argument('--verbose', type=int, default=2)
p.add_argument('--batch_size', type=int, default=32)
p.add_argument('--n_epochs', type=int, default=3)
p.add_argument('--lr', type=float, default=5e-5)
p.add_argument('--warmup_ratio', type=float, default=.2)   
p.add_argument('--adam_epsilon', type=float, default=1e-8)
p.add_argument('--use_radam', action='store_true')   
p.add_argument('--valid_ratio', type=float, default=.2)
p.add_argument('--max_length', type=int, default=100)
config = p.parse_args(args=[])

config.model_fn='models/dacon1_test.pth'
config.train_fn='dataset/train.csv'
config.gpu_id=1
config

Namespace(adam_epsilon=1e-08, batch_size=32, gpu_id=1, lr=5e-05, max_length=100, model_fn='models/dacon1_test.pth', n_epochs=3, pretrained_model_name='beomi/KcELECTRA-base', train_fn='dataset/train.csv', use_radam=False, valid_ratio=0.2, verbose=2, warmup_ratio=0.2)

## 2. 데이터 클리닝
#### 이모티콘, 특수기호, 초성 등 제거(필자는 특수기호와 초성제거는 하지 않음)

In [35]:
import pandas as pd
import re
import emoji
from soynlp.normalizer import repeat_normalize
emojis = ''.join(emoji.UNICODE_EMOJI.keys())
pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-ㅣ가-힣{emojis}]+')
url_pattern = re.compile(
    r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

def clean(x):
    x = pattern.sub(' ', x)
    x = url_pattern.sub('', x)
    #x= re.sub('[-=+#/\^$@*\"※~&%ㆍ』\\‘|\(\)\[\]\<\>`\'…》▷]', '', x)
    #x=re.sub('([ㄱ-ㅎㅏㅑㅓㅕㅛㅣㅐㅔㅒㅖ]+)', '', x)
    x = x.strip()
    x = repeat_normalize(x, num_repeats=2)
    return x

def read_text(fn):
    df=pd.read_csv(fn,sep=',')
    texts=[]
    labels=df['target'].tolist()
    for i in df['reviews']:
        a=clean(str(i))
        texts.append(a)
    return labels, texts

read_text(config.train_fn)

([2,
  1,
  2,
  2,
  5,
  2,
  5,
  2,
  5,
  2,
  5,
  2,
  5,
  4,
  5,
  4,
  2,
  5,
  5,
  1,
  5,
  2,
  5,
  2,
  4,
  5,
  5,
  5,
  2,
  2,
  5,
  5,
  4,
  5,
  2,
  5,
  5,
  5,
  2,
  5,
  1,
  1,
  1,
  5,
  5,
  2,
  1,
  2,
  5,
  2,
  5,
  5,
  2,
  5,
  5,
  2,
  5,
  1,
  1,
  4,
  2,
  5,
  2,
  5,
  1,
  5,
  5,
  5,
  2,
  1,
  4,
  2,
  5,
  2,
  1,
  2,
  5,
  4,
  5,
  5,
  5,
  5,
  2,
  2,
  2,
  5,
  5,
  5,
  4,
  5,
  1,
  5,
  5,
  1,
  5,
  5,
  5,
  1,
  5,
  5,
  1,
  2,
  1,
  2,
  1,
  2,
  5,
  2,
  4,
  2,
  5,
  1,
  5,
  5,
  5,
  5,
  1,
  2,
  1,
  5,
  4,
  1,
  2,
  1,
  5,
  1,
  1,
  2,
  2,
  5,
  5,
  5,
  1,
  1,
  5,
  5,
  4,
  4,
  2,
  5,
  5,
  2,
  1,
  2,
  1,
  1,
  1,
  5,
  2,
  2,
  1,
  2,
  5,
  5,
  2,
  2,
  2,
  5,
  1,
  5,
  5,
  2,
  4,
  2,
  2,
  2,
  1,
  1,
  1,
  4,
  2,
  5,
  4,
  5,
  2,
  4,
  2,
  4,
  4,
  2,
  5,
  5,
  5,
  5,
  2,
  5,
  2,
  4,
  4,
  1,
  5,
  5,
  2,
  2,
  1,
  5,
  5,
  2,
  1,
  5,


## 3. 데이터 증강
#### korEDA를 활용한 데이터 증강

In [36]:
import random
import pickle
import re

wordnet = {}
with open("KorEDA-master/wordnet.pickle", "rb") as f:
	wordnet = pickle.load(f)



def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)

	return parseText


def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break

	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")

	else:
		new_words = ""

	return new_words


def get_synonyms(word):
	synomyms = []

	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass

	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)

	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]

	return new_words

def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)

	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0

	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words

	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words


def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)
	
	return new_words


def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""

		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)



def EDA(sentence, alpha_sr=0, alpha_ri=0, alpha_rs=0.1, p_rd=0.15, num_aug=3):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)

	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1

	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))

	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))

	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))

	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))

	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))

	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)

	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

	augmented_sentences.append(sentence)

	return augmented_sentences

<>:122: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:122: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_2995393/3032537899.py:122: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  words = [word for word in words if word is not ""]


## 4. 데이터 로더
#### train_set, valid_set을 8:2 로 나눈 후 train_set에 대해서만 EDA적용

In [38]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TextClassificationCollator():

    def __init__(self, tokenizer, max_length, with_text=True): 
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.with_text = with_text

    def __call__(self, samples):                       
        texts = [s['text'] for s in samples]   
        labels = [s['label'] for s in samples] 

        encoding = self.tokenizer(            
            texts,
            padding=True,     
            truncation=True,
            return_tensors="pt",       
            max_length=self.max_length,            
            add_special_tokens=True
        )

        return_value = {
            'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],     
            'labels': torch.tensor(labels, dtype=torch.long),
        }
        if self.with_text:
            return_value['text'] = texts

        return return_value


class TextClassificationDataset(Dataset):

    def __init__(self, texts, labels):   
        self.texts = texts
        self.labels = labels
    
    def __len__(self):                   
        return len(self.texts)
    
    def __getitem__(self, item):
        text = str(self.texts[item])    
        label = self.labels[item]      

        return {
            'text': text,
            'label': label,
        }

def get_loaders(fn, tokenizer, valid_ratio=.2):
    labels, texts = read_text(fn)   

    unique_labels = list(set(labels))  
    label_to_index = {}
    index_to_label = {}
    for i, label in enumerate(unique_labels):
        label_to_index[label] = i
        index_to_label[i] = label

  
    labels = list(map(label_to_index.get, labels))   

   
    shuffled = list(zip(texts, labels))  
    random.shuffle(shuffled)
    texts = [e[0] for e in shuffled]
    labels = [e[1] for e in shuffled]
    idx = int(len(texts) * (1 - valid_ratio))
    texts_train=[]
    labels_train=[]
    for i,j in zip(texts[:idx],labels[:idx]):
        a=EDA(i)
        for k in a:
            texts_train.append(k)
            labels_train.append(j)
   
    train_loader = DataLoader(
        TextClassificationDataset(texts_train, labels_train),
        batch_size=config.batch_size,
        shuffle=True,
        collate_fn=TextClassificationCollator(tokenizer, config.max_length),
    )
    valid_loader = DataLoader(   
        TextClassificationDataset(texts[idx:], labels[idx:]),
        batch_size=config.batch_size,
        collate_fn=TextClassificationCollator(tokenizer, config.max_length),
    )

    return train_loader, valid_loader, index_to_label   

from transformers import AutoTokenizer, AdamW, AutoModelForSequenceClassification
tokenizer =AutoTokenizer.from_pretrained(config.pretrained_model_name)  
train_loader, valid_loader, index_to_label = get_loaders(config.train_fn,tokenizer,valid_ratio=config.valid_ratio)
print('|train| =', len(train_loader) * config.batch_size,'|valid| =', len(valid_loader) * config.batch_size)


|train| = 80000 |valid| = 5024


## 5. 모델 설정

In [39]:
import torch.nn as nn
import torch.optim as optim
import torch_optimizer as custom_optim 
from transformers import get_linear_schedule_with_warmup

n_total_iterations = len(train_loader) * config.n_epochs   
n_warmup_steps = int(n_total_iterations * config.warmup_ratio) 

def get_optimizer(model, config):
    if config.use_radam:
        optimizer = custom_optim.RAdam(model.parameters(), lr=config.lr)
    else:
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [{
                'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                'weight_decay': 0.01
            },
            {
                'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                'weight_decay': 0.0
            }]
        optimizer = optim.AdamW(optimizer_grouped_parameters,lr=config.lr, eps=config.adam_epsilon)
    return optimizer


model = AutoModelForSequenceClassification.from_pretrained(config.pretrained_model_name, num_labels=len(index_to_label))
optimizer = get_optimizer(model, config)
crit = nn.CrossEntropyLoss()     
scheduler = get_linear_schedule_with_warmup(optimizer,n_warmup_steps,n_total_iterations)

if config.gpu_id >= 0:
    model.cuda(config.gpu_id)
    crit.cuda(config.gpu_id)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

## 6. ignite.engine 설정
#### best_loss 체크하고 모델 저장

In [40]:
from copy import deepcopy
import numpy as np
import torch
from ignite.engine import Events
from ignite.engine import Engine
from ignite.metrics import RunningAverage
from ignite.contrib.handlers.tqdm_logger import ProgressBar

VERBOSE_SILENT = 0
VERBOSE_EPOCH_WISE = 1
VERBOSE_BATCH_WISE = 2

class EngineForBert(Engine):

    def __init__(self, func, model, crit, optimizer, scheduler, config):
        self.scheduler = scheduler
        self.model = model
        self.crit = crit
        self.optimizer = optimizer
        self.config = config

        super().__init__(func)
        self.best_loss = np.inf
        self.best_model = None

        self.device = next(model.parameters()).device

    @staticmethod
    def train(engine, mini_batch):        
        engine.model.train() 
        engine.optimizer.zero_grad()

        x, y = mini_batch['input_ids'], mini_batch['labels']
        x, y = x.to(engine.device), y.to(engine.device)
        mask = mini_batch['attention_mask']
        mask = mask.to(engine.device)         

        x = x[:, :engine.config.max_length]

        y_hat = engine.model(x, attention_mask=mask).logits      

        loss = engine.crit(y_hat, y)
        loss.backward()

        if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):
            accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))
        else:
            accuracy = 0

        engine.optimizer.step()
        engine.scheduler.step()

        return {'loss': float(loss),'accuracy': float(accuracy)}

    @staticmethod
    def validate(engine, mini_batch):    
        engine.model.eval()

        with torch.no_grad():
            x, y = mini_batch['input_ids'], mini_batch['labels']
            x, y = x.to(engine.device), y.to(engine.device)
            mask = mini_batch['attention_mask']
            mask = mask.to(engine.device)

            x = x[:, :engine.config.max_length]

            y_hat = engine.model(x, attention_mask=mask).logits

            loss = engine.crit(y_hat, y)

            if isinstance(y, torch.LongTensor) or isinstance(y, torch.cuda.LongTensor):
                accuracy = (torch.argmax(y_hat, dim=-1) == y).sum() / float(y.size(0))
            else:
                accuracy = 0

        return {'loss': float(loss),'accuracy': float(accuracy)}

    @staticmethod
    def attach(train_engine, validation_engine, verbose=VERBOSE_BATCH_WISE):
        def attach_running_average(engine, metric_name):          
            RunningAverage(output_transform=lambda x: x[metric_name]).attach(engine,metric_name)                      

        training_metric_names = ['loss', 'accuracy']

        for metric_name in training_metric_names:
            attach_running_average(train_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(train_engine, training_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:
            @train_engine.on(Events.EPOCH_COMPLETED)
            def print_train_logs(engine):
                print('Epoch {} - loss={:.4e} accuracy={:.4f}'.format(
                engine.state.epoch,
                engine.state.metrics['loss'],
                engine.state.metrics['accuracy'],
                ))

        validation_metric_names = ['loss', 'accuracy']
        
        for metric_name in validation_metric_names:
            attach_running_average(validation_engine, metric_name)

        if verbose >= VERBOSE_BATCH_WISE:
            pbar = ProgressBar(bar_format=None, ncols=120)
            pbar.attach(validation_engine, validation_metric_names)

        if verbose >= VERBOSE_EPOCH_WISE:
            @validation_engine.on(Events.EPOCH_COMPLETED)
            def print_valid_logs(engine):
                print('Validation - loss={:.4e} accuracy={:.4f} best_loss={:.4e}'.format(
                    engine.state.metrics['loss'],
                    engine.state.metrics['accuracy'],
                    engine.best_loss,
                ))

    @staticmethod
    def check_best(engine):
        loss = float(engine.state.metrics['loss'])
        if loss <= engine.best_loss: 
            engine.best_loss = loss  
            engine.best_model = deepcopy(engine.model.state_dict()) 

    @staticmethod
    def save_model(engine, train_engine, config, **kwargs):
        torch.save({'model': engine.best_model,'config': config, **kwargs}, config.model_fn)

class BertTrainer():

    def __init__(self,config):  
        self.config = config

    def train(self,model, crit, optimizer, scheduler, train_loader, valid_loader,):                                     
        train_engine = EngineForBert(EngineForBert.train,model, crit, optimizer, scheduler, self.config)
        validation_engine = EngineForBert(EngineForBert.validate,model, crit, optimizer, scheduler, self.config)

        EngineForBert.attach(train_engine,validation_engine,verbose=self.config.verbose)

        def run_validation(engine, validation_engine, valid_loader):
            validation_engine.run(valid_loader, max_epochs=1)

        train_engine.add_event_handler(Events.EPOCH_COMPLETED, run_validation, validation_engine, valid_loader)
        validation_engine.add_event_handler(Events.EPOCH_COMPLETED, EngineForBert.check_best)

        train_engine.run(train_loader,max_epochs=self.config.n_epochs)                                        

        model.load_state_dict(validation_engine.best_model)  

        return model

## 7. 학습, 평가, 저장

In [41]:
trainer = BertTrainer(config)
model = trainer.train(model,crit,optimizer,scheduler,train_loader,valid_loader)
torch.save({'rnn': None,'cnn': None,'bert': model.state_dict(),'config': config,    
    'vocab': None,'classes': index_to_label,'tokenizer': tokenizer}, config.model_fn)

  0%|                                                                                          | 1/2500 [00:00…

Epoch 1 - loss=6.3397e-01 accuracy=0.7410


  1%|5                                                                                          | 1/157 [00:00…

Validation - loss=8.4209e-01 accuracy=0.6655 best_loss=inf


  0%|                                                                                          | 1/2500 [00:00…

...

Validation - loss=1.3312e+00 accuracy=0.6334 best_loss=8.4209e-01


  0%|                                                                                          | 1/2500 [00:00…

Epoch 3 - loss=7.0852e-02 accuracy=0.9791


  1%|5                                                                                          | 1/157 [00:00…

Validation - loss=1.6673e+00 accuracy=0.6367 best_loss=8.4209e-01


## 8. test_set 예측
#### soft_voting을 위해 prob값까지만 출력

In [42]:
import sys, os
import torch.nn.functional as F
file='dataset/test.csv'
def read_text1(file):
    df=pd.read_csv(file,sep=',')
    df_text=df['reviews']
    texts=[]
    for i in df_text:
        a=clean(str(i))
        texts.append(a)   
    return texts

saved_data = torch.load(
    config.model_fn,
    map_location='cpu' if config.gpu_id < 0 else 'cuda:%d' % config.gpu_id)   

train_config = saved_data['config']
bert_best = saved_data['bert']
index_to_label = saved_data['classes']

lines = read_text1(file)

with torch.no_grad():   
    tokenizer = AutoTokenizer.from_pretrained(train_config.pretrained_model_name)
    model_loader =AutoModelForSequenceClassification
    model = model_loader.from_pretrained(
        train_config.pretrained_model_name,
        num_labels=len(index_to_label))
    model.load_state_dict(bert_best)  

    if config.gpu_id >= 0:
        model.cuda(config.gpu_id)
    device = next(model.parameters()).device  

    model.eval()

    y_hats = []
    for idx in range(0, len(lines), config.batch_size):
        mini_batch = tokenizer(
            lines[idx:idx + config.batch_size],
            padding=True,    
            truncation=True,  
            return_tensors="pt")

        x = mini_batch['input_ids']
        x = x.to(device)   
        mask = mini_batch['attention_mask']  
        mask = mask.to(device)

        y_hat = F.softmax(model(x, attention_mask=mask).logits, dim=-1)

        y_hats += [y_hat]
    y_hats = torch.cat(y_hats, dim=0)

    probs, indice = y_hats.cpu().topk(1)

    target1=[]
    target2=[]
    target4=[]
    target5=[]
    for i in y_hats:
        target1.append(float(i[0]))
        target2.append(float(i[1]))
        target4.append(float(i[2]))
        target5.append(float(i[3]))

    df_prob=pd.DataFrame({'1':target1,'2':target2,'4':target4,'5':target5})
    folder_path = os.getcwd()
    df_prob.to_csv('dataset/sample_test.csv',sep='\t',index=False)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

In [43]:
df_prob

,1,2,4,5
0,0.134791,0.836470,0.013698,0.015040
1,0.816368,0.181569,0.001063,0.000999
2,0.001640,0.002421,0.091882,0.904057
3,0.836841,0.160805,0.001123,0.001230
4,0.903871,0.092434,0.001667,0.002027
...,...,...,...,...
24995,0.001938,0.003746,0.173028,0.821289
24996,0.001364,0.001948,0.052843,0.943845
24997,0.571522,0.426592,0.001104,0.000782
24998,0.001788,0.002259,0.055212,0.940741


## 9. soft_voting을 통한 최종 예측값 출력
#### 위의 코드처럼 동일한 방식으로 명령인자만 변경하여 5가지 모델의 예측 확률을 저장 한 csv로 sotf_voting

In [44]:
# 위와 동일한 코드에 명령인자만 교체해서 학습시켜 저장한 모델들을 로드하여 확률값을 예측한 후 csv형태로 저장
# model="monologg/koelectra-base-v3-discriminator" , batch_size=32, epoch=5 >>> result_pro1.csv로 확률값 저장
# model="beomi/KcELECTRA-base" , batch_size=32, epoch=10 >>> result_kc_pro1.csv로 확률값 저장
# model="klue/roberta-base", batch_size=32, epoch=10 >>> result_roberta_pro1.csv로 확률값 저장
# model="beomi/KcELECTRA-base", batch_size=64, epoch=10 , emoji 사용 >>> result_kc_emo_64_pro1.csv로 확률값 저장
# model="beomi/KcELECTRA-base", batch_size=64, epoch=10, emoji 사용, koreda 사용, radam사용, lr=1e-4>>> result_kc_emo4_64_eda_pro1.csv로 확률값 저장
df1=pd.read_csv('dataset/result_pro1.csv',sep='\t')
df2=pd.read_csv('dataset/result_kc_pro1.csv',sep='\t')
df3=pd.read_csv('dataset/result_roberta_pro1.csv',sep='\t')
df4=pd.read_csv('dataset/result_kc_emo_64_pro1.csv',sep='\t')
df5=pd.read_csv('dataset/result_kc_emo4_64_eda_pro1.csv',sep='\t')
df_p1=[]
for i,j,k,l,s in zip(df1['1'],df2['1'],df3['1'],df4['1'],df5['1']):
    a=i+j+k+l+s
    df_p1.append(a)
df_p2=[]
for i,j,k,l,s in zip(df1['2'],df2['2'],df3['2'],df4['2'],df5['2']):
    a=i+j+k+l+s
    df_p2.append(a)
df_p4=[]
for i,j,k,l,s in zip(df1['4'],df2['4'],df3['4'],df4['4'],df5['4']):
    a=i+j+k+l+s
    df_p4.append(a)
df_p5=[]
for i,j,k,l,s in zip(df1['5'],df2['5'],df3['5'],df4['5'],df5['5']):
    a=i+j+k+l+s
    df_p5.append(a)
df_en=pd.DataFrame({'1':df_p1,'2':df_p2,'4':df_p4,'5':df_p5})
df_target=df_en.idxmax(axis=1).tolist()

df_id=pd.read_csv('dataset/test.csv',sep=',')
df_sample=pd.DataFrame({'id':df_id['id'].tolist(),'target':df_target})
df_sample

,id,target
0,0,2
1,1,1
2,2,5
3,3,1
4,4,1
...,...,...
24995,24995,5
24996,24996,5
24997,24997,2
24998,24998,5


## 10. 제출파일 저장

In [61]:
df_sample.to_csv('dataset/sample_submission_en_soft.csv',sep=',',index=False)